In [1]:
# %% [markdown]
# # Entraînement Multi-Modèles
# %%
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier


In [2]:
# %% [markdown]
# ## 1. Chargement et Imputation
# Nous utilisons la régression linéaire pour compléter les données manquantes de `margin_low`.

# 1. Chargement et Imputation (Correction du bug précédent)
df = pd.read_csv("billets.csv", sep=";")
features_reg = ['diagonal', 'height_left', 'height_right', 'margin_up', 'length']
df_train_reg = df.dropna(subset=['margin_low'])
df_pred_reg = df[df['margin_low'].isnull()]

if len(df_pred_reg) > 0:
    reg_imp = LinearRegression()
    reg_imp.fit(df_train_reg[features_reg], df_train_reg['margin_low'])
    df.loc[df['margin_low'].isnull(), 'margin_low'] = reg_imp.predict(df_pred_reg[features_reg])

In [3]:
# 2. Préparation des données (6 variables)
features = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
X = df[features]
y = df['is_genuine']

scaler = StandardScaler()
scaler.fit(X.values)
X_scaled = scaler.transform(X.values)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [4]:
from sklearn.decomposition import PCA

# ... (après avoir créé X_scaled dans votre notebook)

# On entraîne une PCA sur les 2 premières composantes
pca = PCA(n_components=2)
pca.fit(X_scaled)

# Sauvegarde de l'objet PCA
joblib.dump(pca, "pca.pkl")
print("✅ PCA sauvegardée (2 composantes)")

✅ PCA sauvegardée (2 composantes)


In [5]:
# 3. Entraînement des différents modèles
models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_scaled, y)
    joblib.dump(model, f"{name.lower()}.pkl")
    print(f"✅ Modèle {name} sauvegardé sous {name.lower()}.pkl")

✅ Modèle KNN sauvegardé sous knn.pkl
✅ Modèle LogisticRegression sauvegardé sous logisticregression.pkl
✅ Modèle RandomForest sauvegardé sous randomforest.pkl
